### 99 Scala Exercises 10-19

### 10\. Run-length encoding of a list.
Use the result of problem P09 to implement the so-called run-length encoding data compression method. Consecutive duplicates of elements are encoded as tuples (N, E) where N is the number of duplicates of the element E.

In [4]:
// Problem 9 Solution
def pack[A](l: List[A]): List[List[A]] = l match {
    case x::xs => {
        // Split list by predicate. Equivalent to takewhile + dropwhile
        val (ys, zs) = l span(_ == x) 
        ys :: pack(zs)
    }
    case _ => Nil
}

// Problem 10 Solution
// Using 'collect' instead 'map' ensures that the list is not empty
// by using the partial application.
def encode[A](l: List[A]): List[(Int, A)] =
    pack(l) collect { case x::xs => (xs.length + 1, x) }

encode(List('a, 'a, 'a, 'a, 'b, 'c, 'c, 'a, 'a, 'd, 'e, 'e, 'e, 'e))

defined function pack
defined function encode
res3_2: List[(Int, Symbol)] = List((4, 'a), (1, 'b), (2, 'c), (2, 'a), (1, 'd), (4, 'e))

### 11\. Modified run-length encoding.
Modify the result of problem P10 in such a way that if an element has no duplicates it is simply copied into the result list. Only elements with duplicates are transferred as (N, E) terms.

In [5]:
def encodeModified[A](l: List[A]): List[Any] = pack(l) collect {
    case x::xs if xs.length > 0 => (xs.length + 1, x)
    case x::_                   => x
}

encodeModified(List('a, 'a, 'a, 'a, 'b, 'c, 'c, 'a, 'a, 'd, 'e, 'e, 'e, 'e))

defined function encodeModified
res4_1: List[Any] = List((4,'a), 'b, (2,'c), (2,'a), 'd, (4,'e))

### 12\. Decode a run-length encoded list.
Given a run-length code list generated as specified in problem P10, construct its uncompressed version.


In [10]:
def decode[A](l: List[(Int, A)]): List[A] =
    l flatMap { case (n, a) => List.fill(n)(a) }

decode(List((4, 'a), (1, 'b), (2, 'c), (2, 'a), (1, 'd), (4, 'e)))

defined function decode
res9_1: List[Symbol] = List('a, 'a, 'a, 'a, 'b, 'c, 'c, 'a, 'a, 'd, 'e, 'e, 'e, 'e)

### 13\. Run-length encoding of a list (direct solution).
Implement the so-called run-length encoding data compression method directly. I.e. don't use other methods you've written (like P09's pack); do all the work directly.

In [11]:
def encodeDirect[A](l: List[A]): List[(Int, A)] = l match {
    case x::xs => {
        val (ys, zs) = xs span(_ == x)
        (ys.length + 1, x) :: encodeDirect(zs)
    }
    case _ => Nil
}
encodeDirect(List('a, 'a, 'a, 'a, 'b, 'c, 'c, 'a, 'a, 'd, 'e, 'e, 'e, 'e))

defined function encodeDirect
res10_1: List[(Int, Symbol)] = List((4, 'a), (1, 'b), (2, 'c), (2, 'a), (1, 'd), (4, 'e))

In [14]:
// Comparison
time { encodeDirect(List('a, 'a, 'a, 'a, 'b, 'c, 'c, 'a, 'a, 'd, 'e, 'e, 'e, 'e)) }
time { encode(List('a, 'a, 'a, 'a, 'b, 'c, 'c, 'a, 'a, 'd, 'e, 'e, 'e, 'e)) }

res13_0: (List[(Int, Symbol)], concurrent.duration.FiniteDuration) = (List((4, 'a), (1, 'b), (2, 'c), (2, 'a), (1, 'd), (4, 'e)), 139325 nanoseconds)
res13_1: (List[(Int, Symbol)], concurrent.duration.FiniteDuration) = (List((4, 'a), (1, 'b), (2, 'c), (2, 'a), (1, 'd), (4, 'e)), 127253 nanoseconds)

### 14\. Duplicate the elements of a list.

In [14]:
def duplicate[A](l: List[A]): List[A] = l flatMap { x => x::x::Nil }

// Efficient version using local mutable state. Still side effect free.
import scala.collection.mutable._
def duplicateMut[A](l: List[A]): List[A] = {
    val buffer = ListBuffer.empty[A]
    l.foreach { x => buffer += x; buffer += x} 
    buffer.toList
}
// Not relevant difference
val l1 = List('a, 'b, 'c, 'c, 'd)
time { duplicate(l1) }
time { duplicateMut(l1) }
// But...
val l2 = (1 to 3000000).toList
time(duplicate(l2))._2
time(duplicateMut(l2))._2

defined function duplicate
import scala.collection.mutable._

defined function duplicateMut
l1: List[Symbol] = List('a, 'b, 'c, 'c, 'd)
res13_4: (List[Symbol], concurrent.duration.FiniteDuration) = (List('a, 'a, 'b, 'b, 'c, 'c, 'c, 'c, 'd, 'd), 195480 nanoseconds)
res13_5: (List[Symbol], concurrent.duration.FiniteDuration) = (List('a, 'a, 'b, 'b, 'c, 'c, 'c, 'c, 'd, 'd), 178693 nanoseconds)
l2: List[Int] = List(
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
...
res13_7: concurrent.duration.FiniteDuration = 4281170046 nanoseconds
res13_8: concurrent.duration.FiniteDuration = 67388836 nanoseconds

### 15\. Duplicate the elements of a list a given number of times.

In [19]:
// Pure functional way
def duplicateN[A](n: Int)(l: List[A]): List[A] = l flatMap { x => List.fill(n)(x) }

// Using local mutable state. Still side effect free.
import scala.collection.mutable._
def duplicateN_Mut[A](n: Int)(l: List[A]): List[A] = {
    val buffer = ListBuffer.empty[A]
    for (x <- l; _ <- 1 to n) buffer += x
    buffer.toList
}
val l1 = List('a, 'b, 'c, 'c, 'd)
duplicateN(3)(l1)
duplicateN_Mut(3)(l1)
time (duplicateN(5000)(l1))._2
time (duplicateN_Mut(5000)(l1))._2

defined function duplicateN
import scala.collection.mutable._

defined function duplicateN_Mut
l1: List[Symbol] = List('a, 'b, 'c, 'c, 'd)
res18_4: List[Symbol] = List('a, 'a, 'a, 'b, 'b, 'b, 'c, 'c, 'c, 'c, 'c, 'c, 'd, 'd, 'd)
res18_5: List[Symbol] = List('a, 'a, 'a, 'b, 'b, 'b, 'c, 'c, 'c, 'c, 'c, 'c, 'd, 'd, 'd)
res18_6: concurrent.duration.FiniteDuration = 990896 nanoseconds
res18_7: concurrent.duration.FiniteDuration = 1630854 nanoseconds

### 16\. Drop every Nth element from a list.

In [48]:
// Simple recursive method
def dropRec[A](n: Int)(l: List[A]): List[A] = {
    // Inner function to save n between recursive calls.
    def inner(nInner: Int, lInner: List[A]): List[A] = lInner match {
        case _::xs if nInner == 1 => inner(n, xs)
        case x::xs => x :: inner(nInner-1, xs)
        case _ => Nil
    }
    inner(n, l)
}

// Tail recursive method
def dropTailRec[A](n: Int)(l: List[A]): List[A] = {
    // Inner function to save n between recursive calls.
    @annotation.tailrec
    def inner(nInner: Int, lInner: List[A], accum: List[A]): List[A] = lInner match {
        case _::xs if nInner == 1 => inner(n, xs, accum)
        case x::xs => inner(nInner -1,  xs, x +: accum) // prepend takes O(1) and append O(n)
        case _ => accum.reverse
    }
    inner(n, l, List.empty)
}

// Simple but inefficient version 
def drop[A](n: Int)(l: List[A]): List[A] = for {
    (x, i) <- l.zipWithIndex if (i+1) % n != 0
} yield x

val l = List('a, 'b, 'c, 'd, 'e, 'f, 'g, 'h, 'i, 'j, 'k)
drop(3)(l)
dropRec(3)(l)
dropTailRec(3)(l)

val l2 = (1 to 1000).toList
time (drop(3)(l2))._2 // unnecessary zipWithIndex
time (dropRec(3)(l2))._2 // Limited by stack size
time (dropTailRec(3)(l2))._2 // Right implementation

defined function dropRec
defined function dropTailRec
defined function drop
l: List[Symbol] = List('a, 'b, 'c, 'd, 'e, 'f, 'g, 'h, 'i, 'j, 'k)
res47_4: List[Symbol] = List('a, 'b, 'd, 'e, 'g, 'h, 'j, 'k)
res47_5: List[Symbol] = List('a, 'b, 'd, 'e, 'g, 'h, 'j, 'k)
res47_6: List[Symbol] = List('a, 'b, 'd, 'e, 'g, 'h, 'j, 'k)
l2: List[Int] = List(
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
...
res47_8: concurrent.duration.FiniteDuration = 3924436 nanoseconds
res47_9: concurrent.duration.FiniteDuration = 384546 nanoseconds
res47_10: concurrent.duration.FiniteDuration = 246044 nanoseconds

### 17\.  Split a list into two parts.
The length of the first part is given. Use a Tuple for your result.j

In [60]:
// Using Built-in method
def split[A](n: Int)(l: List[A]): (List[A], List[A]) =
    l.splitAt(n)

// Tail recursive solution
def splitTailRec[A](n: Int)(l: List[A]): (List[A], List[A]) = {
    @annotation.tailrec
    def inner(i: Int, innerList: List[A], acc1: List[A],
              acc2: List[A]): (List[A], List[A]) = innerList match {
        case x::xs if i < n => inner(i+1, xs, x +: acc1, acc2)
        case x::xs => inner(i+1, xs, acc1, x +: acc2)
        case _ => acc1.reverse -> acc2.reverse
    }
    inner(0, l, List.empty, List.empty)
}

split(3)(List('a, 'b, 'c, 'd, 'e, 'f, 'g, 'h, 'i, 'j, 'k))
splitTailRec(3)(List('a, 'b, 'c, 'd, 'e, 'f, 'g, 'h, 'i, 'j, 'k))

// Not relevant performance difference 
time { split(3)(List('a, 'b, 'c, 'd, 'e, 'f, 'g, 'h, 'i, 'j, 'k)) }
time { splitTailRec(3)(List('a, 'b, 'c, 'd, 'e, 'f, 'g, 'h, 'i, 'j, 'k)) }

defined function split
defined function splitTailRec
res59_2: (List[Symbol], List[Symbol]) = (List('a, 'b, 'c), List('d, 'e, 'f, 'g, 'h, 'i, 'j, 'k))
res59_3: (List[Symbol], List[Symbol]) = (List('a, 'b, 'c), List('d, 'e, 'f, 'g, 'h, 'i, 'j, 'k))
res59_4: ((List[Symbol], List[Symbol]), concurrent.duration.FiniteDuration) = ((List('a, 'b, 'c), List('d, 'e, 'f, 'g, 'h, 'i, 'j, 'k)), 34112 nanoseconds)
res59_5: ((List[Symbol], List[Symbol]), concurrent.duration.FiniteDuration) = ((List('a, 'b, 'c), List('d, 'e, 'f, 'g, 'h, 'i, 'j, 'k)), 36033 nanoseconds)

## 18\. Extract a slice from a list.
Given two indices, I and K, the slice is the list containing the elements from and including the Ith element up to but not including the Kth element of the original list. Start counting the elements with 0.

In [72]:
// Using Built-in method
def slice[A](i: Int, j: Int)(l: List[A]): List[A] = l.slice(i, j)

// Tail recursive solution
def sliceTailRec[A](i: Int, j: Int)(l: List[A]): List[A] = {
    @annotation.tailrec
    def inner(z: Int, innerList: List[A], accum: List[A]): List[A] =
        innerList match {
            case x::xs if z >= i && z < j => inner(z+1, xs, x +: accum)
            case x::xs if i < j => inner(z+1, xs, accum)
            case _ => accum.reverse
    }
    inner(0, l, List.empty)
}

val l = List('a, 'b, 'c, 'd, 'e, 'f, 'g, 'h, 'i, 'j, 'k)
slice(3, 7)(l)
sliceTailRec(3, 7)(l)

time { slice(3, 7)(l) } // Slightly higher performance
time { sliceTailRec(3, 7)(l) }

defined function slice
defined function sliceTailRec
l: List[Symbol] = List('a, 'b, 'c, 'd, 'e, 'f, 'g, 'h, 'i, 'j, 'k)
res71_3: List[Symbol] = List('d, 'e, 'f, 'g)
res71_4: List[Symbol] = List('d, 'e, 'f, 'g)
res71_5: (List[Symbol], concurrent.duration.FiniteDuration) = (List('d, 'e, 'f, 'g), 8959 nanoseconds)
res71_6: (List[Symbol], concurrent.duration.FiniteDuration) = (List('d, 'e, 'f, 'g), 11963 nanoseconds)

### 19\. Rotate a list N places to the left.

In [82]:
def rotate[A](n: Int)(l: List[A]): List[A] = n match {
    case 0 => l
    case _ => {
        val length = l.length
        val splitIndex = if (n > 0) n % length else (n % length) + length
        val (xs, ys) = l splitAt splitIndex
        ys ++ xs
    }
}

rotate(3)(List('a, 'b, 'c, 'd, 'e, 'f, 'g, 'h, 'i, 'j, 'k))
rotate(-2)(List('a, 'b, 'c, 'd, 'e, 'f, 'g, 'h, 'i, 'j, 'k))

defined function rotate
res81_1: List[Symbol] = List('d, 'e, 'f, 'g, 'h, 'i, 'j, 'k, 'a, 'b, 'c)
res81_2: List[Symbol] = List('j, 'k, 'a, 'b, 'c, 'd, 'e, 'f, 'g, 'h, 'i)